In [1]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" 
!pip install datasets==2.18.0

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
error: externally-managed-environment

× This environment is externally managed
╰

In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
import torch, json, random

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 1024,
    dtype = None,                       # auto-detect
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/run/media/linmar/WORKSPACE/Krishi Sakha - Anywhere, Anytime/finetunenev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 3050 6GB Laptop GPU. Num GPUs = 1. Max memory: 5.668 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 3050 6GB Laptop GPU. Num GPUs = 1. Max memory: 5.668 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore dow

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)

Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
import os 
data_path = os.path.join("datasets", "data.json")
raw = load_dataset("json", data_files=data_path, split="train")

def format_prompt(sample):
    return f"""<start_of_turn>user
{sample['instruction']}
{sample['input']}<end_of_turn>
<start_of_turn>model
{sample['output']}<end_of_turn>"""

ds = raw.map(lambda x: {"text": format_prompt(x)}, remove_columns=raw.column_names)

In [ ]:
import os
import torch

# Prevent torch._dynamo from being available to unsloth
os.environ["TORCHDYNAMO_DISABLE"] = "1"
os.environ["TORCH_DYNAMO_DISABLE"] = "1"

# Disable graph breaks and force eager execution
torch._dynamo.config.suppress_errors = True
torch._dynamo.config.disable = True

steps = (len(ds) * 3) // (2 * 8) + 1   # 3 epochs, bs=2, grad_acc=8
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds,
    dataset_text_field = "text",
    max_seq_length = 1024,
    args = dict(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 50,
        max_steps = steps,            # ≈ 375
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        output_dir = "gov_en2ml_lora",
        optim = "adamw_8bit",
        save_steps = steps // 3,
    ),
)
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30,000 | Num Epochs = 3 | Total steps = 11,250
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 13,045,760 of 1,012,931,712 (1.29% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30,000 | Num Epochs = 3 | Total steps = 11,250
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 13,045,760 of 1,012,931,712 (1.29% trained)


Step,Training Loss
1,9.271400
2,9.630200
3,9.653100
4,9.213900
5,9.550200
6,9.321000
7,9.842700
8,9.603600
9,9.506200
10,9.271900


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
model.save_pretrained_merged("gemma3-gov-en2ml", tokenizer, save_method="merged_16bit")
# 2 GB safetensors ready

In [ ]:
from transformers import pipeline
import collections.abc # Import collections.abc for checking sequence type

pipe = pipeline("text-generation",
                model="gemma3-gov-en2ml",
                tokenizer=tokenizer,
                device_map="auto")

def translate_to_malayalam(english_input):
    if isinstance(english_input, collections.abc.Sequence) and not isinstance(english_input, str):
        # Handle list of strings
        prompts = [f"""<start_of_turn>user
Translate to Malayalam.
{s}
<end_of_turn>
<start_of_turn>model
""" for s in english_input]

        outputs = pipe(prompts,
                       max_new_tokens=80,
                       temperature=0.3,
                       do_sample=True,
                       pad_token_id=tokenizer.eos_token_id)

        translated_texts = []
        for i, output_item in enumerate(outputs):
            translated_texts.append(output_item[0]["generated_text"][len(prompts[i]):])
        return translated_texts
    else:
        # Handle single string
        prompt = f"""<start_of_turn>user
Translate to Malayalam.
{english_input}
<end_of_turn>
<start_of_turn>model
"""
        out = pipe(prompt,
                   max_new_tokens=80,
                   temperature=0.3,
                   do_sample=True,
                   pad_token_id=tokenizer.eos_token_id)
        return out[0]["generated_text"][len(prompt):]

# Example usage for single string:
# english_text = "The weather is very nice today."
# malayalam_text = translate_to_malayalam(english_text)
# print(malayalam_text)

# Example usage for a list of strings:
# english_texts = ["Hello world", "How are you?"]
# malayalam_texts = translate_to_malayalam(english_texts)
# for eng, mal in zip(english_texts, malayalam_texts):
#    print(f"English: {eng} -> Malayalam: {mal}")

In [ ]:
english_text = "The weather is very nice today."
malayalam_text = translate_to_malayalam(english_text)
print(malayalam_text)

In [ ]:
english_texts = [
    "The weather is very nice today.",
    "She likes to read books every evening.",
    "I forgot my phone at home.",
    "The children are playing in the garden.",
    "He is learning a new language.",
    "The government has announced new policies to improve education.",
    "Many tourists visit this place during the winter season.",
    "The meeting was postponed due to unexpected issues.",
    "I couldn’t attend the event because of a traffic jam.",
    "Scientists are working on a new technology to reduce pollution.",
    "Despite the challenges, the team successfully completed the project on time.",
    "The doctor explained that early diagnosis can significantly improve the chances of recovery.",
    "Several communities protested against the proposal, claiming it would damage the environment.",
    "The report highlights the importance of digital literacy in modern workplaces.",
    "Although the company faced financial losses last year, it is slowly recovering."
]

malayalam_texts = translate_to_malayalam(english_texts)

for eng, mal in zip(english_texts, malayalam_texts):
    print(f"English: {eng}")
    print(f"Malayalam: {mal}\n")

In [ ]:
# Interactive Chat with Fine-tuned Model
def chat_with_model(user_message):
    """Chat with the fine-tuned Gemma model"""
    prompt = f"""<start_of_turn>user
    answer in english
{user_message}
<end_of_turn>
<start_of_turn>model
"""
    response = pipe(prompt,
                   max_new_tokens=150,
                   temperature=0.7,
                   do_sample=True,
                   pad_token_id=tokenizer.eos_token_id)
    return response[0]["generated_text"][len(prompt):]

# Example: Chat with the model
user_input = "hello"
result = chat_with_model(user_input)
print(f"User: {user_input}")
print(f"Model: {result}")